psi_1 & ~psi_2

Logical AND of psi_1 with the negation of psi_2. ~psi_1 | omega

Logical OR between the negation of psi_1 and omega. (psi_1 & omega) | (~psi_1 & ~omega)

A logical expression equivalent to an XOR between psi_1 and omega. (psi_1 | psi_2) & ~omega

Logical AND between the result of the OR operation on psi_1 and psi_2, and the negation of omega. psi_1 ^ psi_2 ^ omega

The logical XOR operation applied pairwise among psi_1, psi_2, and omega.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import networkx as nx
import sympy as sp

# Define symbols for sympy expressions
psi_1, psi_2 = sp.symbols('psi_1 psi_2')
phi, omega = sp.symbols('phi omega')

# Functions f1 and f2
def f1(theta):
    if theta == 0:
        return np.pi / 2
    return np.arcsin(np.sin(theta)) + (np.pi / 2) * (1 - np.pi / (2 * theta))

def f2(theta):
    if theta == 0:
        return np.pi / 2
    return np.arcsin(np.cos(theta)) + (np.pi / 2) * (1 - np.pi / (2 * theta))

# Define decision graph for logic modulation
def create_decision_graph():
    G = nx.DiGraph()
    logical_nodes = ['psi_1', 'psi_2', 'omega', 'phi']
    for node in logical_nodes:
        # Initialize logical nodes with a value in the range [0, 1]
        G.add_node(node, value=np.random.rand())
    return G

# Function to update the logic values with user-entered formula
def update_with_user_input(G, user_input):
    # Convert the entered user formula to a sympy expression
    try:
        formula = sp.sympify(user_input, locals=globals())

        # Now, evaluate the formula using the current values in the graph
        substitutions = {str(var): G.nodes[str(var)]['value'] for var in formula.free_symbols}
        result = formula.subs(substitutions).evalf()

        # Update the 'phi' value in the graph
        G.nodes['phi']['value'] = float(result)
        print(f"Graph updated with logic formula: {user_input}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Function to visualize the decision graph as a tessellation pattern
def visualize_tessellation(G, domain, hex_centers, hex_size):
    fig, ax = plt.subplots()
    for center in hex_centers:
        x, y = center
        theta = np.arctan2(y, x) if x != 0 else np.pi / 2
        efficiency_value = f1(theta) * f2(theta) * (1 - f1(theta)) * (1 - f2(theta))
        logic_value = G.nodes['phi']['value']
        color_value = np.clip(efficiency_value * logic_value, 0, 1)
        hexagon = patches.RegularPolygon(center, numVertices=6, radius=hex_size, orientation=np.pi/6)
        hexagon.set_facecolor(plt.cm.viridis(color_value))
        ax.add_patch(hexagon)
    ax.set_xlim(domain[0], domain[1])
    ax.set_ylim(domain[2], domain[3])
    ax.set_aspect('equal')
    plt.axis('off')
    plt.show()

# Initialize domain and hex centers for the tessellation
domain = (-5, 5, -5, 5)
hex_size = 0.5
hex_centers = [(i, j) for i in np.arange(domain[0], domain[1], hex_size)
               for j in np.arange(domain[2], domain[3], hex_size)]

# Main interaction loop for the game with user input
def run_game_loop():
    G = create_decision_graph()
    while True:
        user_input = input("Enter your logic formula (e.g., 'psi_1 & ~psi_2') or 'quit' to exit: ")
        if user_input.lower() == 'quit':
            print("Exiting game.")
            break
        update_with_user_input(G, user_input)
        visualize_tessellation(G, domain, hex_centers, hex_size)

run_game_loop()